## **Using Frontier Models to Optimise a Python Code by converting it into C++ Code !!**

In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr 
import subprocess
from ollama import Client

In [2]:
load_dotenv(override=True)

openai = OpenAI()
OPENAI_MODEL = 'gpt-4.1-mini'

client = Client()
OLLAMA_MODEL='llama3.1:8b'

#### Building System and User Messages

In [3]:
system_message = "You are an assistant that reimplements Python code in high performance C++ code for an Windows 11 x64 PC."
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments."
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [4]:
def user_promt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time."
    user_prompt += "Respond only with C++ code; do not explain your work other thanb a few comments."
    user_prompt += "Pay attention to the number types to ensure no int overflows. Remember to #include all necessary C++ packages such as <iomanip>.\n\n"
    user_prompt += python
    return user_prompt

In [5]:
def messages_for(python):
    return [
        {'role': 'system', 'content': system_message}, 
        {'role': 'user', 'content': user_promt_for(python)}
    ]

In [6]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp", "").replace("```", "")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [7]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end="", flush=True)
    write_output(reply)

In [8]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
exec(pi)

Result: 3.141592658589
Execution Time: 10.436005 seconds


In [48]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int64_t iterations, int64_t param1, int64_t param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i * param1 - param2);
        result -= 1.0 / j;
        j = static_cast<double>(i * param1 + param2);
        result += 1.0 / j;
    }
    return result;
}

int main() {
    const int64_t iterations = 100000000;
    const int64_t param1 = 4;
    const int64_t param2 = 1;

    auto start = std::chrono::high_resolution_clock::now();
    
    double result = calculate(iterations, param1, param2) * 4.0;
    
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> diff = end - start;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << "\n";
    std::cout << "Execution Time: " << diff.count() << " seconds\n";

    return 0;
}
```

#### **See the difference !!**

In [ ]:
# running the python code

exec(pi)

Result: 3.141592658589
Execution Time: 10.516519 seconds


In [49]:
# running the optimized c++ code !!

!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe


Result: 3.141592658589
Execution Time: 0.202601600000 seconds


In [50]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [51]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 38.317430 seconds


In [58]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <cstdint>
#include <iomanip>
#include <chrono>
#include <algorithm>

class LCG {
    uint64_t value;
    static constexpr uint64_t a = 1664525;
    static constexpr uint64_t c = 1013904223;
    static constexpr uint64_t m = (1ULL << 32);
public:
    explicit LCG(uint64_t seed) : value(seed) {}
    uint32_t next() {
        value = (a * value + c) % m;
        return static_cast<uint32_t>(value);
    }
};

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = static_cast<int>(lcg.next() % range) + min_val;
    }
    // Kadane's algorithm for O(n)
    int64_t max_sum = static_cast<int64_t>(random_numbers[0]);
    int64_t current_sum = 0;
    for (int i = 0; i < n; ++i) {
        current_sum += random_numbers[i];
        if (current_sum > max_sum)
          

In [59]:
# running the optimized c++ code !!

!g++ -O3 -std=c++17 -march=native -o optimized optimized.cpp
!optimized.exe

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.000433 seconds


#### **Time For Some UI !!**

In [60]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n', '').replace('```','')

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python Code", lines=10)
        cpp = gr.Textbox(label="C++ Code", lines=10)
    with gr.Row():
        model=gr.Dropdown(["GPT"], label="Select Model", value="GPT")
        convert = gr.Button("Convert Code")
    
    convert.click(stream_gpt, inputs=[python], outputs=[cpp])
ui.launch(inbrowser=True)

In [74]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()


In [75]:
def execute_cpp(code):
    write_output(code)
    try:
        compile_cmd = [
            "g++", "-Ofast", "-std=c++17", "-march=native",
            "-o", "optimized.exe", "optimized.cpp"
        ]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        
        run_cmd = ["optimized.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"



In [76]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [79]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(stream_gpt, inputs=[python], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
